In [1]:
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score
import re
import numpy as np

np.random.seed(512)

In [2]:
def is_url(row, reg, pat):
    samples = row[12:17]
    for sample in samples:
        if reg.match(str(sample).strip()) and len(re.sub(pat, '', str(sample).strip())) == 0:
            return True
    return False


def is_datetime(row):
    samples = row[12:17]
    for sample in samples:
        try:
            _ = pd.Timestamp(sample)
            return True
        except ValueError:
            try:
                _ = pd.Timestamp(int(sample))
                return True
            except ValueError:
                continue
    return False


def is_email(row, reg):
    samples = row[12:17]
    for sample in samples:
        if reg.match(str(sample)):
            return True
    return False


def is_num(row, reg):
    samples = row[12:17]
    for sample in samples:
        try:
            _ = float(str(sample))
            return True
        except ValueError:
            if reg.match(str(sample).lower()):
                return True
    return False

url_pat = r"(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?"
url_reg = re.compile(url_pat)

list_pat =  r"((\d|\w|\')+(,|>|;|:|\-|`\.|\||\*){1}\s?(\d|\w|\')+){2,}"
list_reg = re.compile(list_pat)

num_pat = r"([\$|\w]?(\d|\d\.\d|\d\,\d)+\s?[\$|\w]?)"
num_reg = re.compile(num_pat)

email_pat = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,6}\b"
email_reg = re.compile(email_pat)

In [3]:
testdf = pd.read_csv('../../Benchmark-Labeled-Data/data_test.csv')
test_metadata = pd.read_csv('../../Benchmark-Labeled-Data/Metadata/meta_data.csv')

dict_label_true = {
    'numeric': 0,
    'categorical': 1,
    'datetime': 2,
    'sentence': 3,
    'url': 4,
    'embedded-number': 5,
    'list': 6,
    'not-generalizable': 7,
    'context-specific': 8
}

test_merged = pd.merge(testdf,test_metadata,on='Record_id')
y_true = test_merged.y_act.values.tolist()
y_true = [dict_label_true[str(i)] for i in y_true]

In [4]:
def get_col_dtype(col):
        
    if col.dtype =="object":
        try:
            col_new = pd.to_numeric(col.dropna().unique())
            return 'Numeric'
        except:
            return "Object"
    if col.dtype =="bool": return "Object"
    else:
        if col.dtype == 'float64' or col.dtype == 'int64':
            return 'Numeric'
        else:      
            return col.dtype

In [5]:
predictions = []
y_truth_iter = []
data = test_merged
prv_csv_name = ''

for index, data in test_merged.iterrows():
    if index%100==0: print(index)

    s1 = data['sample_1']
    avg_tokens = len(str(s1).split(' '))
    
    nans = data['%_nans']
    dist_val = data['%_dist_val']
    num_dist_val = data['num_of_dist_val']

    mean_wd_cnt = data['mean_word_count']
    std_wd_cnt = data['std_dev_word_count']
    has_delims =  data['has_delimiters']
    col = data['Attribute_name']
    
    csv_name = '../../RawCSV/' + str(data['name'])
    
    
    if prv_csv_name != csv_name:  df = pd.read_csv(csv_name,encoding='latin1')     
        
    try: df_col = df[col]
    except:
        predictions.append(1)
        continue    
    
    
    curtype = get_col_dtype(df_col)
    
    if str(curtype) == 'Numeric':
        y_truth_iter.append(data['y_act'])
        
        if nans > 99.99: 
            predictions.append(7)
            continue

        if num_dist_val < 10 :
            y_truth_iter.append(data['y_act'])
            predictions.append(7)
        else:
            predictions.append(0)
            continue
    else:
        
        if nans > 99.99:
            y_truth_iter.append(data['y_act'])
            predictions.append(7)
            continue

        if dist_val > 99.99:
            y_truth_iter.append(data['y_act'])
            predictions.append(7)
            continue
            
        if num_dist_val < 100 :
            y_truth_iter.append(data['y_act'])
            predictions.append(1)
        else:
            y_truth_iter.append(data['y_act'])
            
            if float(nans) >= 0.90:
                predictions.append(8)
            elif float(mean_wd_cnt) <= 2.0:
                if is_datetime(data):
                    predictions.append(2)
                elif is_email(data, email_reg):
                    predictions.append(8)
                elif is_url(data, url_reg, url_pat):
                    predictions.append(4)
                elif is_num(data, num_reg):
                    predictions.append(5)
                else:
                    predictions.append(8)
            elif float(std_wd_cnt) < 10.0:
                if has_delims == True:
                    predictions.append(6)
                else:
                    predictions.append(8)
            else:
                predictions.append(3)                
            continue
        

0


/tmp/ipykernel_277890/2060328343.py:24: DtypeWarning: Columns (41,142) have mixed types. Specify dtype option on import or set low_memory=False.
  if prv_csv_name != csv_name:  df = pd.read_csv(csv_name,encoding='latin1')
/tmp/ipykernel_277890/2060328343.py:24: DtypeWarning: Columns (41,142) have mixed types. Specify dtype option on import or set low_memory=False.
  if prv_csv_name != csv_name:  df = pd.read_csv(csv_name,encoding='latin1')
/tmp/ipykernel_277890/2060328343.py:24: DtypeWarning: Columns (41,142) have mixed types. Specify dtype option on import or set low_memory=False.
  if prv_csv_name != csv_name:  df = pd.read_csv(csv_name,encoding='latin1')
/tmp/ipykernel_277890/2060328343.py:24: DtypeWarning: Columns (41,142) have mixed types. Specify dtype option on import or set low_memory=False.
  if prv_csv_name != csv_name:  df = pd.read_csv(csv_name,encoding='latin1')
/tmp/ipykernel_277890/2060328343.py:24: DtypeWarning: Columns (41,142) have mixed types. Specify dtype option on

100


/tmp/ipykernel_277890/2060328343.py:24: DtypeWarning: Columns (403,405,832,833,841,844,857,915,916,928,929,941,942,954,955,957,958,967,968,970,971,980,981,993,994,996,997,1006,1007,1009,1010,1022,1023,1035,1036,1048,1049,1071,1072,1084,1085,1087,1088,1097,1098,1100,1101,1110,1111,1113,1114,1126,1127,1139,1140,1149,1150,1152,1153,1162,1163,1165,1166,1178,1179,1191,1192,1204,1205,1217,1218,1227,1228,1230,1231,1240,1241,1243,1244,1256,1257,1269,1270,1282,1283,1295,1296,1306,1308,1309,1319,1321,1322,1335,1347,1348,1373,1374,1386,1387,1399,1400,1412,1413,1451,1452,1453,1480,1481,1482,1534,1535,1536,1537,1538,1539,1540,1541,1542,1543,1544,1545,1546,1547,1548,1549,1654,1669,1704,1732,1734,1735,1736,1737,1738,1739,1740,1741,1742,1743,1744,1746,1747,1748,1749,1750,1751,1752,1753) have mixed types. Specify dtype option on import or set low_memory=False.
  if prv_csv_name != csv_name:  df = pd.read_csv(csv_name,encoding='latin1')
/tmp/ipykernel_277890/2060328343.py:24: DtypeWarning: Columns (403,

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score


def PrintMetrics(y_true, y_pred):
    print(dict_label_true)
    print(f'Accuracy: {accuracy_score(y_true, y_pred)}')

    matrix = confusion_matrix(y_true, y_pred)
    print(matrix)

    precision = precision_score(y_true, y_pred, average=None)
    recall = recall_score(y_true, y_pred, average=None)
    f1 = f1_score(y_true, y_pred, average=None)

    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 score: {f1}')


PrintMetrics(y_true, predictions)

{'numeric': 0, 'categorical': 1, 'datetime': 2, 'sentence': 3, 'url': 4, 'embedded-number': 5, 'list': 6, 'not-generalizable': 7, 'context-specific': 8}
Accuracy: 0.5390428211586902
[[669   0   0   0   0   1   0  37   0]
 [ 52 209   8   0   0   2   1 128  57]
 [  4   7  19   0   0   0   1  90  20]
 [  0  25   0   4   0   0   2  24  37]
 [  0  12   0   0   8   0   0   6   6]
 [  0  35   4   0   0  18   0  37   5]
 [  1   7   0   0   0   0   0  43   6]
 [ 35  59   0   0   0   2   0 109  10]
 [105   8   3   0   0   1   3  31  34]]
Precision: [0.77251732 0.57734807 0.55882353 1.         1.         0.75
 0.         0.21584158 0.19428571]
Recall: [0.94625177 0.45733042 0.13475177 0.04347826 0.25       0.18181818
 0.         0.50697674 0.18378378]
F1 score: [0.85060394 0.51037851 0.21714286 0.08333333 0.4        0.29268293
 0.         0.30277778 0.18888889]


In [ ]:
# {'numeric': 0, 'categorical': 1, 'datetime': 2, 'sentence': 3, 'url': 4, 'embedded-number': 5, 'list': 6, 'not-generalizable': 7, 'context-specific': 8}
# Accuracy: 0.5390428211586902
# [[669   0   0   0   0   1   0  37   0]
#  [ 52 209   8   0   0   2   1 128  57]
#  [  4   7  19   0   0   0   1  90  20]
#  [  0  25   0   4   0   0   2  24  37]
#  [  0  12   0   0   8   0   0   6   6]
#  [  0  35   4   0   0  18   0  37   5]
#  [  1   7   0   0   0   0   0  43   6]
#  [ 35  59   0   0   0   2   0 109  10]
#  [105   8   3   0   0   1   3  31  34]]
# Precision: [0.77251732 0.57734807 0.55882353 1.         1.         0.75
#  0.         0.21584158 0.19428571]
# Recall: [0.94625177 0.45733042 0.13475177 0.04347826 0.25       0.18181818
#  0.         0.50697674 0.18378378]
# F1 score: [0.85060394 0.51037851 0.21714286 0.08333333 0.4        0.29268293
#  0.         0.30277778 0.18888889]